In [1]:
#importing libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 100000 #setting row boundary length
pd.options.display.max_columns = 100000 #setting column bundary length

In [3]:
#Reading S3 bucket data, i.e. CSV file
bucket='adl-core-sagemaker-studio/external/User/csv_from_edh'
data_key = 'Data_13Aug2021.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
print(data_location)
df=pd.read_csv(data_location)

s3://adl-core-sagemaker-studio/external/User/csv_from_edh/Data_13Aug2021.csv


KeyboardInterrupt: 

In [ ]:
df=pd.read_csv('s3://adl-core-sagemaker-studio/external/User/csv_from_edh/Data_13Aug2021.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (86,121,129,197) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

In [13]:
#display shape of the dataframe
df.shape

(247794, 208)

In [14]:
df = df.drop_duplicates()

In [18]:
df.count()

rnk                                                       247794
contactid                                                 247794
transcript                                                247794
cnt                                                       247794
call_dur_sec                                              247794
client_key                                                247794
implementation_key                                        247794
agent_key                                                 247794
participant_key                                           247794
team_key                                                  247794
team_id                                                   247794
access_direct_session_id                                       0
service_center_interaction_id                             224134
agent_id                                                  247794
person_id                                                 247794
person_internal_id       

In [19]:
#Reading S3 bucket data, i.e. CSV file
bucket='adl-core-sagemaker-studio/external/metadata/call_data_with_agent_id_joins'
data_key = 'call_topic_mapping.csv'
data_location1 = 's3://{}/{}'.format(bucket, data_key)
print(data_location)
df1=pd.read_csv(data_location1)

s3://adl-core-sagemaker-studio/external/User/csv_from_edh/Data_13Aug2021.csv


In [20]:
df1.shape

(13435, 4)

In [21]:
df1.head()

,CSPRO_PRIMARY_TOPIC/SECONDARY DRIVER,CSPRO_SUBTOPIC/TERTIARY DRIVER,PRIMARY DRIVER,business_process_domain
0,ACCESS,COULD NOT AUTHENTICATE,Authentication,CIM
1,ACCESS,IVR/AD ISSUES,Authentication,CIM
2,ACCESS,PASSWORD ISSUES,Authentication,CIM
3,ACCESS,SECURITY QUESTIONS ISSUES,Authentication,CIM
4,ACCESS,SUBTOPIC -CALLER NOT AUTHORIZED,Authentication,CIM


In [22]:
df1.rename(columns = {'CSPRO_PRIMARY_TOPIC/SECONDARY DRIVER':'cspro_primary_topic',
                            'CSPRO_SUBTOPIC/TERTIARY DRIVER':'cspro_primary_subtopic',
                           'PRIMARY DRIVER':'primary driver'},inplace=True)

In [23]:
df1.head()

,cspro_primary_topic,cspro_primary_subtopic,primary driver,business_process_domain
0,ACCESS,COULD NOT AUTHENTICATE,Authentication,CIM
1,ACCESS,IVR/AD ISSUES,Authentication,CIM
2,ACCESS,PASSWORD ISSUES,Authentication,CIM
3,ACCESS,SECURITY QUESTIONS ISSUES,Authentication,CIM
4,ACCESS,SUBTOPIC -CALLER NOT AUTHORIZED,Authentication,CIM


In [24]:
df['cspro_primary_topic'] = df['cspro_primary_topic'].str.upper()
df['cspro_primary_subtopic'] = df['cspro_primary_subtopic'].str.upper()
df1['primary driver'] = df1['primary driver'].str.upper()
df['cspro_primary_topic'] = df['cspro_primary_topic'].str.strip()
df['cspro_primary_subtopic'] = df['cspro_primary_subtopic'].str.strip()
df1['primary driver']=df1['primary driver'].str.strip()
df1['cspro_primary_topic']=df1['cspro_primary_topic'].str.strip()
df1['cspro_primary_subtopic']=df1['cspro_primary_subtopic'].str.strip()

In [25]:
df = pd.merge(df, df1[['cspro_primary_topic','cspro_primary_subtopic','primary driver']], 
                    how="left", 
                    left_on=['cspro_primary_topic','cspro_primary_subtopic'], 
                    right_on=['cspro_primary_topic','cspro_primary_subtopic']);

In [30]:
df.shape

(269158, 209)

In [26]:
df['primary driver'].nunique()

110

In [27]:
df['primary driver'].value_counts()

REFERRALS/TRANSFERS              45270
SPENDING ACCOUNTS                24789
ANNUAL ENROLLMENT                20954
COBRA                            12779
ENROLLMENT                        9874
MEDICAL                           9819
DB&P                              9499
CASE MANAGEMENT                   8390
LIFE EVENTS                       6498
COMMUNICATIONS                    6294
AUTHENTICATION                    6194
CLAIMS/DISPUTES/APPEALS           5927
PERSONAL DATA CHANGES             5835
PLAN PROVISIONS/PROCEDURES        5742
RETIREMENT                        5735
CALL CONNECTION ISSUES            5266
DEPENDENT VERIFICATION            4281
NAVIGATION/TROUBLESHOOTING        3899
DISTRIBUTIONS                     3699
OFS                               2879
LIFE INSURANCE                    2831
DB COMMENCEMENT                   2785
LEAVES                            2614
MEDICARE                          2483
OTHER BENEFITS/PROGRAMS           2471
DEATH                    

In [15]:
#picking only valid intents
# df = df[df['cspro_primary_topic'].isin(valid_intents)]

In [31]:
df.shape

(269158, 209)

In [33]:
#delete duplicate records, considering only 2 columns for check uniqueness
# data = df
data = df.drop_duplicates(subset=['transcript','primary driver'], keep="first")

In [34]:
#shape/dimension of csv after removing duplicates
data.shape

(250647, 209)

In [42]:
data['primary driver'].value_counts()

REFERRALS/TRANSFERS              41111
SPENDING ACCOUNTS                22810
ANNUAL ENROLLMENT                19306
COBRA                            10898
ENROLLMENT                        9616
MEDICAL                           9418
DB&P                              9332
CASE MANAGEMENT                   7882
LIFE EVENTS                       6060
AUTHENTICATION                    6006
COMMUNICATIONS                    5974
PERSONAL DATA CHANGES             5640
PLAN PROVISIONS/PROCEDURES        5606
CLAIMS/DISPUTES/APPEALS           5578
RETIREMENT                        5455
CALL CONNECTION ISSUES            4984
DEPENDENT VERIFICATION            3993
NAVIGATION/TROUBLESHOOTING        3784
DISTRIBUTIONS                     3647
OFS                               2741
LIFE INSURANCE                    2668
DB COMMENCEMENT                   2658
LEAVES                            2557
OTHER BENEFITS/PROGRAMS           2391
DEATH                             2363
MEDICARE                 

In [37]:
#display the transcript data
data['transcript'].head()

0    B'\n\n\n\nagent: \t \ngood morning thank you f...
1    B'\n\n\n\ncustomer: \t \nthank you for calling...
2    B'\n\n\n\nagent: \t \nthank you for calling ex...
3    B'\n\ncustomer: \t \nthank you actually for de...
4    B'\n\n\n\nagent: \t \ngood morning thank you f...
Name: transcript, dtype: object

In [38]:
#removing first 2 letters and last letter from the transcript
data['transcript'] = data['transcript'].str[2:-1]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
#display the transcript after removing the first 2 and last etter
data['transcript'].head()

0    \n\n\n\nagent: \t \ngood morning thank you for...
1    \n\n\n\ncustomer: \t \nthank you for calling a...
2    \n\n\n\nagent: \t \nthank you for calling exte...
3    \n\ncustomer: \t \nthank you actually for deep...
4    \n\n\n\nagent: \t \ngood morning thank you for...
Name: transcript, dtype: object

In [45]:
# total intents in the dataset
data['primary driver'].nunique()

110

In [46]:
#total intents having frequency less than 50
data[data.groupby('primary driver')['primary driver'].transform('size') > 500]['primary driver'].nunique()

63

In [47]:
#code to display total count of intents(column 2) on the basis of their frequencies(column 1)
vc_df = pd.DataFrame(data['primary driver'].value_counts())
vc_df.iloc[:,0].groupby(vc_df.iloc[:,0]).count().head(10)

primary driver
1     2
2     1
3     1
6     1
13    2
15    1
20    1
21    1
22    1
23    1
Name: primary driver, dtype: int64

In [48]:
# data = data[pd.notnull(data['cspro_primary_topic'])]
# print(data['full_transcript'].head(10))
# print(data['full_transcript'].apply(lambda x: len(x.split(' '))).sum())

In [49]:
#import nltk libraries
import nltk
import re
from nltk.stem.porter import PorterStemmer #used for stemming
from nltk.stem import WordNetLemmatizer #used for lemmatization
from nltk.tokenize import sent_tokenize #used for sentence tokenization
ps = PorterStemmer()
wordnet=WordNetLemmatizer()

In [116]:
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords.append('.')
lst_stopwords.append('thank')
lst_stopwords.append('thanks')
lst_stopwords.append('would')
lst_stopwords.append('could')
lst_stopwords.append('should')
lst_stopwords.append('must')
lst_stopwords.append('might')
lst_stopwords.append('redacted')
lst_stopwords.append('agent')
lst_stopwords.append('customer')
lst_stopwords.append('um')
lst_stopwords.append('umm')
lst_stopwords.append('hm')
lst_stopwords.append('hmm')
lst_stopwords.append('yes')
lst_stopwords.append('yeah')
lst_stopwords.append('yep')
lst_stopwords.append('hello')
lst_stopwords.append('hey')
lst_stopwords.append('hi')
lst_stopwords.append('bye')
lst_stopwords.append('yeah')
lst_stopwords.append('oh')
lst_stopwords.append('sir')
lst_stopwords.append('okay')
lst_stopwords.append('huh')
lst_stopwords.append('uh')
lst_stopwords.append('er')
lst_stopwords.append('e')
lst_stopwords.append('x')
lst_stopwords.append('also')
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [134]:
#function for data preprocessing
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #stopwords hardcoded, need to remove going furthur
    text = text.replace('\\nagent:','. ').replace('\\ncustomer:','. ').replace('\\n','').replace('\\t','').replace('`',"'")
    sentences = nltk.sent_tokenize(text)
    corpus = []
    for i in range(len(sentences)):
        script = re.sub('[^a-zA-Z]', ' ', sentences[i])
        script = script.lower()
        script = script.split()
        script = [wordnet.lemmatize(word) for word in script if not word in set(lst_stopwords)]
        if script:
            script = ' '.join(script)
            corpus.append(script)
    return corpus

In [135]:
clean_text(data['transcript'][0])

['good morning calling benefit center speaking ruby holman',
 'ruby question increase contribution health saving account',
 'happy assist like process easy question',
 'know wanna hide like',
 'provide last four digit social please',
 'name account',
 'scott mcdonald',
 'much like',
 'run dollar paycheck pay pay gert month',
 'currently contribution dollar hundred dollar paycheck safely double tooth dollar per paycheck',
 'great one moment',
 'double date give tooth dollar per pay period let',
 'fine fine',
 'alrighty new begin date february',
 'great possible online call',
 'able online option change contribution',
 'go ahead sorry',
 'log benefit website think life side computer change chance eight contribution',
 'love',
 'help go health benefit website',
 'dad eight com click manager mary center',
 'give option know pay option change agency contribution',
 'benefit sender alright check next time alright got',
 'good',
 'much',
 'already',
 'good day']

In [137]:
# #cleaning the whole dataset using the above logic
data['transcript'] = data['transcript'].apply(clean_text)
# print_plot(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [138]:
#exporting the data to csv file because of memory issue in AWS
data.to_csv('dataset_after_cleaning_transcript.csv',index=False)

In [30]:
# intents_to_exclude = data[data.groupby('cspro_primary_topic')['cspro_primary_topic'].transform('size') < 4]['cspro_primary_topic']

In [31]:
# y = data['cspro_primary_topic']
# X = data['full_transcript']

In [32]:
#train_test_split method

In [33]:
# balanced_data = data[data.groupby('cspro_primary_topic')['cspro_primary_topic'].transform('size') > 4]['cspro_primary_topic']

In [34]:
# print(data['full_transcript'].apply(lambda x: len(x.split(' '))).sum())